<p align="center">
  12 de Novembro de 2021
</p>

<br>
<br>

<p align="center">
  <img height="150" src="https://www.ccs.ufscar.br/imagens/ufscar-preto.png">
</p>

<p align="center">
  <img height="150" src="https://site.dc.ufscar.br/static/media/LOGO-DC.295bfc37.svg">
</p>

---

<h1 align="center">Trabalho de Aprendizado de Máquina</h1>


<h3 align="center">Profº. Dr. Diego Furtado Silva</h3>


<p align="center">
  Bárbara Dib Oliveira (769717) <br>
  Igor Teixeira Machado (769708) <br>
  Lucas Machado Cid (769841) <br>
  Vinicius Quaresma da Luz (769836)
</p>

---

Inicialmente, importamos as bibliotecas necessárias.

In [1]:
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
import csv

In [2]:
ranking: list = [];

def add_to_rank(entry: map):
    ranking.append(entry)

    # Ordena o ranking com base na chave 'score'
    ranking.sort(key=lambda x: x['score'], reverse=True)

Carrega o dataset e divide-o em treinamento e teste.

In [4]:
file = open('datasets/student_data_por.csv')
csvreader = csv.reader(file)
header = next(csvreader)
data = []

for row in csvreader:
  data.append(row)

X = []
Y = []

# Pega os dados crus e os divide em atributos e classes.
for i in range(len(data)):
    l: list = list(data[i])

    # Os 3 últimos atributos são as notas.
    X.append(l[:-3])

    # Nota final.
    score = l[-1]

    numericScore = int(score)

    # Obs.: As notas vão até 20.
    if(numericScore < 10):
      gradeRange = 0
    else:
      gradeRange = 1
      
    Y.append(gradeRange)

Usa um `OneHotEncoder` para transformar as variáveis categóricas em variáveis numéricas.

In [5]:
enc = OneHotEncoder(handle_unknown='ignore');
X = enc.fit_transform(X)

Faz a padronização dos dados.

:warning: Ainda que a padronização não provoque alteração alguma no Grid Search do Decision Tree, ela faz com que a acurácia do kNN seja igual à acurácia do Decision Tree.

In [6]:
scaler = StandardScaler(with_mean=False).fit(X)
X = scaler.transform(X)

Divide os dados em conjuntos de treinamento e teste.

In [7]:
x_tr, x_te, y_tr, y_te = train_test_split(X, Y, test_size = 0.2, random_state=42, stratify=Y)

# Grid Search para Decision Tree

In [8]:
classifier = DecisionTreeClassifier()
param_grid = {'criterion': ['entropy', 'gini'],
              'max_depth': range(2,20,2),
              'min_samples_leaf': range(2,10,2),
              'min_impurity_decrease': np.linspace(0,0.5,10)}
gs = GridSearchCV(classifier, param_grid=param_grid)

gs.fit(x_tr, y_tr)
y_pred = gs.predict(x_te)

gs_dt_acc = accuracy_score(y_te, y_pred)
add_to_rank({'classifier': 'Decision Tree', 'score': gs_dt_acc})

print('Acurácia obtida: ', gs_dt_acc)


Acurácia obtida:  0.8923076923076924


# Faz um Grid Search para o kNN

In [9]:
classifier = KNeighborsClassifier()

classifier.fit(x_tr, y_tr)

y_pred = classifier.predict(x_te)
accuracy_score(y_te, y_pred)

# Grid Search with classifier
param_grid = {'weights': ['uniform', 'distance'],
              'n_neighbors': range(1,15,2),
              'metric': ['euclidean', 'manhattan']}
gs = GridSearchCV(classifier, param_grid=param_grid)

gs.fit(x_tr, y_tr)
y_pred = gs.predict(x_te)

gs_knn_acc = accuracy_score(y_te, y_pred)

print('Acurácia obtida: ', gs_knn_acc)
add_to_rank({'classifier': 'kNN', 'score': gs_knn_acc})

if gs_knn_acc > gs_dt_acc:
  print('Houve melhoria de ', gs_knn_acc - gs_dt_acc)
else:
    print('Houve piora de ', gs_dt_acc - gs_knn_acc)

Acurácia obtida:  0.8846153846153846
Houve piora de  0.007692307692307776


# Usando Random Forest

In [9]:
# Importa o random forest
from sklearn.ensemble import RandomForestClassifier

# Calcula o random forest
classifier = RandomForestClassifier()

# Usa o classifier para treinar
classifier.fit(x_tr, y_tr)

# Imprime a acurácia
y_pred = classifier.predict(x_te)

rf_acc = accuracy_score(y_te, y_pred)

# imprime os resultados
print('Acurácia obtida: ', rf_acc)
add_to_rank({'classifier': 'Random Forest', 'score': rf_acc})

Acurácia obtida:  0.8153846153846154


# Naive Bayes

In [10]:
# Calcula naive bayes com os dados padronizados
classifier = GaussianNB()
classifier.fit(x_tr.toarray(), y_tr)

accuracy_score(y_te, classifier.predict(x_te.toarray()))

0.2076923076923077

# Comparação dos métodos usados

In [11]:
print(ranking)

# Prints a ranking of classifiers showing difference to previous
for i in range(len(ranking)):
    if i == 0:
        print(str(i+1) + ' - ' + ranking[i]['classifier'] + ': ' + str(ranking[i]['score']))
    else:
        print(str(i+1) + ' - ' + ranking[i]['classifier'] + ': ' + str(ranking[i]['score'] - ranking[i-1]['score']))
    

[{'classifier': 'Random Forest', 'score': 0.8153846153846154}, {'classifier': 'Decision Tree', 'score': 0.7923076923076923}, {'classifier': 'kNN', 'score': 0.7846153846153846}]
1 - Random Forest: 0.8153846153846154
2 - Decision Tree: -0.023076923076923106
3 - kNN: -0.007692307692307665
